In [1]:
from pathlib import Path
from openai import OpenAI
import io
from pydub import AudioSegment
from pydub.playback import play
import os
from dotenv import load_dotenv

In [2]:
# env variables
load_dotenv()
my_key = os.getenv('OPENAI_API_KEY')

# OpenAI API
client = OpenAI(api_key=my_key)

### Creating the Thread

In [3]:
# create a thread
thread = client.beta.threads.create()
print("created thread: ", thread.id)

created thread:  thread_VyoqRMiKYzgkuyc5FdX2cspl


### Adding messages to thread

In [4]:
# add a message to a thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [5]:
# print thread messages
thread_messages = client.beta.threads.messages.list(message.thread_id)
#print(thread_messages.data[0].content[0].text.value)
print(thread_messages.data) # Does this have an assistant id?

[ThreadMessage(id='msg_cUrofbY1NWdG3Q9NkVcpxbIn', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699582492, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_VyoqRMiKYzgkuyc5FdX2cspl')]


### Creating and running the assistant

In [6]:
# Creating the assistant
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

In [7]:
# Add the assistant to the thread. I.e We're running the assistant to the thread 
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
#print(run)

In [8]:
print(run.status)

queued


In [9]:
# Display the assistant's response.
#The response we get back is information about the run.
#For example the status of the run, the assistant's response, and the assistant's response time.
assistant_response = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

# Get information about the run.
print(assistant_response)

Run(id='run_X1m9wa1v0KaxbpphkxzdYbNA', assistant_id='asst_T6xM35vnmZjOy1pfGIcXrKHe', cancelled_at=None, completed_at=1699582512, created_at=1699582507, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1699582507, status='completed', thread_id='thread_VyoqRMiKYzgkuyc5FdX2cspl', tools=[ToolAssistantToolsCode(type='code_interpreter')])


In [10]:
type(assistant_response)

openai.types.beta.threads.run.Run

In [11]:
# retrieve the Messages added by the Assistant to the Thread
thread_messages = client.beta.threads.messages.list(message.thread_id)
type(thread_messages.data) # prints list
# unpack the list
#for message in thread_messages.data:
#    print(message.content)
#print(thread_messages.data[0].content)
# print type of content
#type(thread_messages.data[0].content) # prints list
# print what is in the list
#print(thread_messages.data[0].content[0].text.value)



list

In [12]:
print(thread_messages.data) # Contains details about both the user and assistant messages

[ThreadMessage(id='msg_AkhwokTtfYabDUuoNRkzh9hV', assistant_id='asst_T6xM35vnmZjOy1pfGIcXrKHe', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\).'), type='text')], created_at=1699582511, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_X1m9wa1v0KaxbpphkxzdYbNA', thread_id='thread_VyoqRMiKYzgkuyc5FdX2cspl'), ThreadMessage(id='msg_cUrofbY1NWdG3Q9NkVcpxbIn', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699582492, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_VyoqRMiKYzgkuyc5FdX2cspl')]


In [13]:
# Get the user message
print("Your question: 👇")
print(thread_messages.data[1].content[0].text.value)

Your question: 👇
I need to solve the equation `3x + 11 = 14`. Can you help me?


In [14]:
# Get the assistant's response
print("Assistant response is:")
print(thread_messages.data[0].content[0].text.value)

Assistant response is:
The solution to the equation \(3x + 11 = 14\) is \(x = 1\).


### New interaction

- Create a new message
- Invoke a new instance of assistant on the thread

In [15]:
# add a message to a thread
new_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please explain your reasoning"
)

In [16]:
# New invokation of the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [17]:
print(run.status)

queued


In [18]:
thread_messages = client.beta.threads.messages.list(new_message.thread_id)
# Get the assistant's response
#print("Assistant response is:")
print(thread_messages.data[0])

ThreadMessage(id='msg_UIyFx6b2IB4yF4h0pxFdCkys', assistant_id='asst_T6xM35vnmZjOy1pfGIcXrKHe', content=[MessageContentText(text=Text(annotations=[], value='Sure! To solve the equation \\(3x + 11 = 14\\), we follow these steps:\n\n1. We start with the given equation:\n   \\[3x + 11 = 14\\]\n\n2. Our goal is to isolate \\(x\\) on one side of the equation to find its value. To do this, we can start by subtracting \\(11\\) from both sides of the equation to get rid of the constant term on the same side as \\(x\\). This gives us:\n   \\[3x + 11 - 11 = 14 - 11\\]\n   Simplifying this results in:\n   \\[3x = 3\\]\n\n3. Now we have \\(3x\\) equals \\(3\\). The next step is to divide both sides by \\(3\\) to solve for \\(x\\):\n   \\[\\frac{3x}{3} = \\frac{3}{3}\\]\n   Simplifying, we find:\n   \\[x = 1\\]\n\nSo the solution to the equation is \\(x = 1\\). This value of \\(x\\) will satisfy the original equation. We can verify this by plugging the solution back into the original equation. If bo

In [19]:
# Get the assistant's response
print("Assistant response is:")
print(thread_messages.data[0].content[0].text.value)

Assistant response is:
Sure! To solve the equation \(3x + 11 = 14\), we follow these steps:

1. We start with the given equation:
   \[3x + 11 = 14\]

2. Our goal is to isolate \(x\) on one side of the equation to find its value. To do this, we can start by subtracting \(11\) from both sides of the equation to get rid of the constant term on the same side as \(x\). This gives us:
   \[3x + 11 - 11 = 14 - 11\]
   Simplifying this results in:
   \[3x = 3\]

3. Now we have \(3x\) equals \(3\). The next step is to divide both sides by \(3\) to solve for \(x\):
   \[\frac{3x}{3} = \frac{3}{3}\]
   Simplifying, we find:
   \[x = 1\]

So the solution to the equation is \(x = 1\). This value of \(x\) will satisfy the original equation. We can verify this by plugging the solution back into the original equation. If both sides are equal after substitution, then the solution is correct:

\[3(1) + 11 = 3 + 11 = 14\]

Hence, \(x = 1\) is the correct solution to the equation \(3x + 11 = 14\).
